In [2]:
with open('text.txt', 'r', encoding="utf-8") as f:
    text = f.read()

In [3]:
print('lenth of dataset in characters: ', len(text))

lenth of dataset in characters:  11853750


In [4]:
#Printing first 100 characters
print(text[:100])

Albert of Prussia ( 17 May 1490 20 March 1568 ) was the last Grand Master of the Teutonic Knights , 


In [5]:
#All unique chr in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz{
86


In [6]:
#mapping of char to int
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # str to int list
decode = lambda l: ''.join([itos[i] for i in l]) #int to str

print(encode("Hello"))
print(decode(encode("Hello")))

[40, 63, 70, 70, 73]
Hello


In [7]:
#Encoding entire text in dataset and storing in a torch.tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print (data.shape,data.type)
print(data[:100])

torch.Size([11853750]) <built-in method type of Tensor object at 0x000002D4AFE730B0>
tensor([33, 70, 60, 63, 76, 78,  1, 73, 64,  1, 48, 76, 79, 77, 77, 67, 59,  1,
         8,  1, 17, 23,  1, 45, 59, 83,  1, 17, 20, 25, 16,  1, 18, 16,  1, 45,
        59, 76, 61, 66,  1, 17, 21, 22, 24,  1,  9,  1, 81, 59, 77,  1, 78, 66,
        63,  1, 70, 59, 77, 78,  1, 39, 76, 59, 72, 62,  1, 45, 59, 77, 78, 63,
        76,  1, 73, 64,  1, 78, 66, 63,  1, 52, 63, 79, 78, 73, 72, 67, 61,  1,
        43, 72, 67, 65, 66, 78, 77,  1, 12,  1])


In [8]:
#split of data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([33, 70, 60, 63, 76, 78,  1, 73, 64])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([33]) the target: 70
when input is tensor([33, 70]) the target: 60
when input is tensor([33, 70, 60]) the target: 63
when input is tensor([33, 70, 60, 63]) the target: 76
when input is tensor([33, 70, 60, 63, 76]) the target: 78
when input is tensor([33, 70, 60, 63, 76, 78]) the target: 1
when input is tensor([33, 70, 60, 63, 76, 78,  1]) the target: 73
when input is tensor([33, 70, 60, 63, 76, 78,  1, 73]) the target: 64


In [11]:
torch.manual_seed(1337)
batch_size = 4 #independent seq process in parallel
block_size = 8 #max length for prediction

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):         #batch dimension
    for t in range(block_size):     #time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()} the target: {target}')

inputs:
torch.Size([4, 8])
tensor([[14,  1, 40, 63,  1, 59, 78, 78],
        [80, 59,  1, 14,  1, 52, 66, 63],
        [81, 59, 77,  1, 72, 73, 78,  1],
        [67, 73, 72, 59, 70, 63,  1,  8]])
targets:
torch.Size([4, 8])
tensor([[ 1, 40, 63,  1, 59, 78, 78, 63],
        [59,  1, 14,  1, 52, 66, 63,  1],
        [59, 77,  1, 72, 73, 78,  1, 73],
        [73, 72, 59, 70, 63,  1,  8,  1]])
----
when input is [14] the target: 1
when input is [14, 1] the target: 40
when input is [14, 1, 40] the target: 63
when input is [14, 1, 40, 63] the target: 1
when input is [14, 1, 40, 63, 1] the target: 59
when input is [14, 1, 40, 63, 1, 59] the target: 78
when input is [14, 1, 40, 63, 1, 59, 78] the target: 78
when input is [14, 1, 40, 63, 1, 59, 78, 78] the target: 63
when input is [80] the target: 59
when input is [80, 59] the target: 1
when input is [80, 59, 1] the target: 14
when input is [80, 59, 1, 14] the target: 1
when input is [80, 59, 1, 14, 1] the target: 52
when input is [80, 59, 1, 1